In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

## Funkcje pomocnicze

In [2]:
# Zmienia format daty z sekund na rrrr-mm-dd
def dataformat(data):
    new_data = time.localtime(data)
    if new_data.tm_mday<10:
        data_day = '0'+str(new_data.tm_mday)
    else:
        data_day = str(new_data.tm_mday)
        
    if new_data.tm_mon<10:
        data_mon = '0' + str(new_data.tm_mon)
    else:
        data_mon =  str(new_data.tm_mon)

    g_data = str(new_data.tm_year) + '-' +data_mon + '-' + data_day
    
    return(g_data)
    

In [3]:
# generuje id, first-pierwsza wartosc, second-druga, i -koncowka
def id_generator(i, first = 0, second= 0):
    if i<10:
        n_id = str(first)+str(second)+'00'+str(i)
    elif i<100 and i >=10 :
        n_id = str(first)+str(second)+'0'+str(i)
    elif i<1000 and i>=100:
        n_id = str(first)+str(second)+str(i)
    return(n_id)

## Klasa zwierzę

In [4]:
class Animal:
    def __init__(self, species):
        
        self.species = species
        
        if self.species == 'pies':
            self.years = 15
            #sample from normal distribution with mean 6 and variance 3 - age of species
            self.mi = 4
            self.sigma = 3
            
        elif self.species == 'kot':
            self.years = 16
            self.mi = 2
            self.sigma = 4
            
        elif self.species == 'szczur':
            self.years = 2.5
            self.mi = 1.25
            self.sigma = 0.5
        self.wizyty = []
        self.daty_wizyt = []
        self.ceny = []
    def stan_zdrowia(self):
        ost_data_wiz = self.daty_wizyt[-1]
        if ost_data_wiz+7*24*3600 < self.actual_time:
            rand = np.random.random()
            if rand > 0.1:
                stan_zdr = "zdrowy"
            else:
                stan_zdr = "chory"
        else:
            stan_zdr = "w czasie leczenia" 
            
        self.stan_zdr = stan_zdr
        return(self.stan_zdr)
        
    def Wizyty(self, ad_time = None): 
        
        ost_data_wizyty = self.daty_wizyt[-1]
        choroby = ["przeziębienie", "zapalenie uszu","świeżb", "zapalenie spojówek","skaleczenie", "inne"]
        if ad_time is not None:
            data_wiz = np.random.uniform(ost_data_wizyty+24*3600, ad_time-24*3600)
        else:
            data_wiz = np.random.uniform(ost_data_wizyty+24*3600, self.actual_time-24*3600)
        zalec = "lek"
        self.daty_wizyt.append(data_wiz)
        chor = choroby[int(np.random.uniform(0, len(choroby)))]
        self.wizyty.append(chor)
        cena = int(np.round(np.random.uniform(50, 150),0))
        self.ceny.append(cena)
        return(data_wiz, chor, zalec, cena) 
    
    def choroby(self):
        
        return(self.wizyty, self.daty_wizyt, self.ceny)
    
    def Species(self):
        if self.species == "pies":
            self.species_id = 1
        elif self.species == "kot":
            self.species_id = 2
        elif self.species == "szczur":
            self.species_id = 3
            
        return(self.species_id)
    
    def plec(self):
        
        rand = np.random.random()
        if rand > 0.5:
            plec = "samica"
        else:
            plec = "samiec"
            
        self.plec = plec
        return(plec)
            
    def name(self, file):
        
        df = pd.read_csv(file, names = ['samce', 'samice'], encoding='unicode_escape')
        df = df.loc[:503]
        
        if self.plec == "samica":
            df = df['samice']
        elif self.plec == "samiec":
            df = df['samce']
        
        rand = np.round(np.random.uniform(0,len(df)-1),0)
        name = (df.values)[int(rand)]
        self.name = name
        
        return(self.name)
        
    def umaszczenie(self):
        
        list_colours = ['biały', 'czarny', 'szary', 'rudy','brązowy', 'biało-czarny', 'biszkoptowy', 'biało-rudy', 'biało-brązowy','biało-szary','inne']
        len_colours = len(list_colours)
        random_c = np.round(np.random.uniform(0,len_colours-1), 0)
        umaszczenie = list_colours[int(random_c)]
        
        self.umaszczenie = umaszczenie
        
        return(self.umaszczenie, self.species) 

    
    def birth(self, death_year = 0):
        
        years2sec = 365.25*24*3600
        
        self.actual_time = time.time() - death_year*years2sec  #actual time in second)
        begin_time = self.actual_time - self.years*years2sec #years- how many years back (in seconds)
        sec_mi = self.mi*years2sec
        sec_std = self.sigma*years2sec
        birth_date = 0
        
        #looking for dates beetwen begin and actual time
        while begin_time > birth_date or self.actual_time < birth_date:
        
            birth_date  = np.random.normal(begin_time+sec_mi, (sec_std), 1)
            
        #birth date of animal    
        self.birth_date = birth_date
        
        return( self.birth_date )
        
    def masa(self):
        #sprawdzam date urodzenia bo pies majacy 2 miesiace nie bedzie wazyl 40kg
        act = (time.localtime(self.actual_time))
        bir = (time.localtime(self.birth_date))
        if bir.tm_year == act.tm_year:
            if abs(bir.tm_mon-act.tm_mon) < 6:
                if self.species == 'pies':
                    masa = np.random.normal(5,5)
                    while masa < 0.5:
                        masa = np.random.normal(5,5)
                elif self.species == 'kot':
                    masa = np.random.normal(1,1)
                    while masa < 0.4 or masa > 2:
                        masa = np.random.normal(1,1)
                elif self.species == 'szczur':
                    masa = np.random.normal(0.1,0.2)
                    while masa < 0.1 or masa > 0.45:
                        masa = np.random.normal(0.1,0.2)
                
            else:
                if self.species == 'pies':
                    masa = np.random.normal(12,6)
                    while masa < 0.5 or masa > 30:
                        masa = np.random.normal(12,6)
                elif self.species == 'kot':
                    masa = np.random.normal(1,1.3)
                    while masa < 0.6 or masa > 3:
                        masa = np.random.normal(1,1.3)
                elif self.species == 'szczur':
                    masa = np.random.normal(0.2,0.2)
                    while masa < 0.24 or masa > 0.55:
                        masa = np.random.normal(0.2,0.2)
        else:
            if self.species == 'pies':
                masa = np.random.normal(17,7)
                while masa < 1 or masa > 45:
                    masa = np.random.normal(17,7)
            elif self.species == 'kot':
                masa = np.random.normal(3, 2)
                while masa < 1 or masa > 5:
                    masa = np.random.normal(3, 2)
            elif self.species == 'szczur':
                masa = np.random.normal(0.25,0.2)
                while masa < 0.24 or masa > 0.7:
                    masa = np.random.normal(0.25,0.2)
        self.masa  = np.round(masa, 3)

        return(self.masa)
    
    def acceptance(self, o_years = 3.21): #3.21 years - opening of pound
        
        years2sec = 365.25*24*3600
        open_time = self.actual_time - o_years*years2sec
        
        acc_time = self.actual_time+100
        
        while acc_time > self.actual_time or bool(acc_time < self.birth_date+50*24*3600):
#             print(acc_time > self.actual_time)
#             print(bool(acc_time < self.birth_date))
            acc_time = np.random.uniform(open_time, self.actual_time) 
            
        #print(time.localtime(acc_time))
        #acceptance date    
        self.acc_time = acc_time
        p_wizyta = acc_time+24*3600
        self.daty_wizyt.append(p_wizyta)
        self.wizyty.append("pierwsza wizyta")
        self.ceny.append(50)
        return( self.acc_time )
    
    def adoption(self):
        
        day2sec = 24*3600
        ad_time = np.random.uniform(self.acc_time+7*day2sec, self.actual_time-14*day2sec) 
          
        self.ad_time = ad_time
        #self.adoptujacy()
        
        return( self.ad_time )
    
#print(acc_time > self.actual_time)
#print(bool(acc_time < self.birth_date))
#print(time.localtime(acc_time))
#acceptance date  
    def adoptujacy(self, own_names, own_lastnames, own_streets, own_city):
        own_ind0 = int(np.round(np.random.uniform(0,len(own_names)),0))
        own_ind1 = int(np.round(np.random.uniform(0,len(own_lastnames)),0))
        own_ind2 = int(np.round(np.random.uniform(0,len(own_streets)),0))
        own_name = own_names[own_ind0][0]
        own_lastname = own_lastnames[own_ind1][0]
        own_street = own_streets[own_ind2]+' ' + str(int(np.random.uniform(1,100)))
        own_tel = str(int(np.random.uniform(100,999)))+"-"
        own_tel = own_tel + str(int(np.random.uniform(100,999)))+"-"+str(int(np.random.uniform(100,999)))
        owner = {}
        owner['name'] = own_name
        owner['last_name'] = own_lastname
        owner['adres'] = own_street + ' ' + own_city
        owner['telefon'] = own_tel
        
        return(owner)
            
        
    def kastracja(self):
        rand = np.random.random()
        if rand > 0.1:
            self.kastracja = "tak"
            n_rand = np.random.random()
            if n_rand > 0.5:
                self.k_day = self.acc_time+ np.random.uniform(10,35)*24*3600
                self.daty_wizyt.append(self.k_day)
                self.wizyty.append("kastracja")
                self.ceny.append(200)
            else:
                self.k_day = self.birth_date + np.random.uniform(200,350)*24*3600

        else:
            self.kastracja = "nie"
            self.k_day = 0
        
        
        return(self.kastracja, self.k_day)
    def next_acceptance(self):
        day2sec = 24*3600
        new_acc_time = 0
        #20% zwierzakow wraca do schorniska
        if self.ad_time is not None:
            new_acc_time = np.random.uniform(self.ad_time+3*day2sec, self.actual_time-7*day2sec ) 
        #print(acc_time > self.actual_time)
        #print(bool(acc_time < self.birth_date))
        #print(time.localtime(acc_time))
        #acceptance date    
        self.acc_time = new_acc_time
        p_wizyta = self.acc_time+24*3600
        self.daty_wizyt.append(p_wizyta)
        self.wizyty.append("pierwsza wizyta")
        self.ceny.append(50)
        return( self.acc_time )
    
    
#sterylizacja - klasa zwierze - data wizty +1 dzien +tydzien sterylizacja
#0.01 0.01
#2 lata
#aktu -200
# wykladniczy ale od pewnego momentu z paramtrem 1
#masa ciała
#400 - 95% adotptowane , 5% zmarłe
#adopcje
# maks - 5, srednio -2, procent wracajacych - 30% 
# w wiekszosci darowizna, co miesiac -wyplaty, dotacja z urzedu
#imiona - nie powtarzaja sie - usuwam z listy uzyte imie


## Funkcja pracownik

In [5]:
def pracownik(own_names, own_lastnames, own_streets, own_city, years=3.21):
        own_ind0 = int(np.round(np.random.uniform(0,len(own_names)),0)-1)
        own_ind1 = int(np.round(np.random.uniform(0,len(own_lastnames)),0)-1)
        own_ind2 = int(np.round(np.random.uniform(0,len(own_streets)),0)-1)
        own_name = own_names[own_ind0][0]
        own_lastname = own_lastnames[own_ind1][0]
        own_street = own_streets[own_ind2]+' ' + str(int(np.random.uniform(1,100)))
        own_tel = str(int(np.random.uniform(100,999)))+"-"
        own_tel = own_tel + str(int(np.random.uniform(100,999)))+"-"+str(int(np.random.uniform(100,999)))
        owner = {}
        owner['name'] = own_name
        owner['last_name'] = own_lastname
        owner['adres'] = own_street + ' ' + own_city
        owner['telefon'] = own_tel
        years2sec = 365.25*24*3600
        actual_time = time.time()
        open_time = actual_time - years*years2sec
        
        data_zat = np.random.uniform(open_time, actual_time)
        data_zw =  np.random.uniform(data_zat, actual_time+24*3600)
        owner["zatr"] = data_zat
        owner["zwoln"] = data_zw
        return(owner)

In [6]:
ulice = ['ul. Agrestowa',
'ul. Akacjowa',
'ul. Armii Krajowej',
'ul. Balonowa',
'ul. Batalionów Chłopskich',
'ul. gen. Józefa Bema',
'ul. Błękitna',
'ul. Bociania',
'ul. Jerzego Bocka',
'ul. Boczna',
'ul. Bolesława Krzywoustego',
'ul. Arkadiusza Bożka',
'ul. Bratnia',
'ul. Brzozowa',
'ul. Ceglana',
'ul. Fryderyka Chopina',
'ul. Cicha',
'ul. Ciepła',
'ul. Antoniego Cieszyńskiego',
'ul. Cmentarna',
'ul. Bronisława Czecha',
'ul. Ignacego Daszyńskiego',
'ul. Henryka Dąbrowskiego',
'ul. Demokratów',
'ul. Kazimierza Deyny',
'ul. Dobroszycka',
'ul. Energetyczna',
'ul. Fabryczna',
'ul. Gliniana',
'ul. Władysława Grabskiego',
'ul. Gryczana',
'ul. gen. Józefa Hallera',
'ul. św. Jadwigi',
'ul. Jagodowa',
'ul. Jaśminowa',
'ul. Jarzynowa',
'ul. Jasna',
'ul. Jastrzębia',
'ul. Jęczmienna',
'ul. Kasztanowa',
'ul. Kazimierza Wielkiego',
'ul. Jana Kilińskiego',
'ul. gen. Franciszka Kleeberga',
'ul. Klonowa',
'ul. Jana Kochanowskiego',
'ul. Kolejowa',
'ul. Kościelna',
'ul. Tadeusza Kościuszki',
'skwer Kombatantów Rzeczypospolitej',
'ul. Mikołaja Kopernika',
'ul. Kosynierów',
'ul. Krótka',
'ul. Kresowa',
'ul. Krucza',
'ul. Krzywa',
'ul. Janusza Kusocińskiego',
'ul. Eugeniusza Felicjana Kwiatkowskiego',
'ul. Kwiatowa',
'ul. Joachima Lelewela',
'ul. Leśna',
'ul. Leszczynowa',
'ul. Bolesława Limanowskiego',
'ul. Lipowa',
'ul. 11 Listopada',
'ul. Lotnicza',
'ul. Stefana Żeromskiego',
'ul. Żwirki i Wigury',
'ul. Żytnia',
'ul. Ludwikowska',
'ul. Lwowska',
'ul. Henryka Łasaka',
'ul. Łąkowa',
'ul. Łowiecka',
'ul. Łukanowska',
'ul. Łużycka',
'pl. Książąt Śląskich',
'ul. Tadeusza Ślusarskiego',
'ul. Świerkowa',
'ul. gen. Stanisława Maczka',
'ul. 3 Maja',
'ul. Malinowa',
'ul. Bronisława Malinowskiego',
'ul. Małopolna',
'ul. Jana Matejki',
'ul. Adama Mickiewicza',
'ul. Stanisława Mikołajczyka',
'ul. Miła',
'ul. Miodowa',
'ul. Młynarska',
'ul. Mokra',
'ul. Ignacego Mościckiego',
'ul. Stanisława Moniuszki',
'ul. Na Podkowie',
'ul. Gabriela Narutowicza',
'ul. Nowowiejska',
'ul. Ogrodowa',
'ul. Okrężna',
'ul. Orla',
'ul. Osada Bystre',
'ul. Osiedle',
'ul. Otwarta',
'ul. Owocowa',
'ul. Owsiana',
'ul. Ignacego Paderewskiego',
'ul. Parkowa',
'ul. Północna',
'ul. Pocztowa',
'ul. Podchorążych',
'ul. Pogodna',
'ul. Polna',
'ul. Południowa',
'ul. Księcia Józefa Poniatowskiego',
'ul. Poprzeczna',
'ul. Porzeczkowa',
'ul. Powietrzna',
'ul. Poziomkowa',
'ul. Prosta',
'ul. Bolesława Prusa',
'ul. Przemysłowa',
'ul. Kazimierza Przerwy-Tetmajera',
'ul. Przyjaźni',
'ul. Bogusława Psujka',
'ul. Pszeniczna',
'ul. Ptasia',
'ul. Kazimierza Pułaskiego',
'ul. Heleny Radlińskiej',
'ul. Różana',
'ul. Mikołaja Reja',
'ul. Tadeusza Rejtana',
'ul. Władysława Stanisława Reymonta',
'rondo Rondo Jana Pawła II',
'rondo Rondo Miast Partnerskich',
'ul. Wandy Rutkiewicz',
'ul. Rycerska',
'ul. Rzemieślnicza',
'ul. Rzepakowa',
'ul. Sejmowa',
'ul. Janusza Sidły',
'ul. Henryka Sienkiewicza',
'ul. gen. Władysława Sikorskiego',
'ul. Jana Sinapiusa',
'ul. Marii Skłodowskiej-Curie',
'ul. Słoneczna',
'ul. Juliusza Słowackiego',
'ul. Sokola',
'ul. Ignacego Solarza',
'rondo Solidarności',
'ul. Sosnowa',
'ul. Spacerowa',
'ul. Spadochronowa',
'ul. Spokojna',
'ul. Feliksa Stamma',
'ul. Jana Stapińskiego',
'pl. Stanisława Staszica',
'ul. Stawowa',
'ul. Stolarska'
'ul. mjr. Henryka Sucharskiego',
'ul. ks. Franciszka Sudoła',
'ul. Szkolna',
'ul. Szybowcowa',
'ul. Targowa',
'ul. Tęczowa',
'ul. Lwa Tołstoja',
'ul. Trakcyjna',
'ul. Romualda Traugutta',
'ul. Stanisławy Walasiewicz',
'ul. Wałowa',
'ul. Wały Jagiellońskie',
'ul. Wądoły',
'ul. Wąska',
'ul. Wesoła',
'ul. Wiejska',
'ul. Wielkopolna',
'ul. Wierzbowa',
'ul. Wikliniarska',
'ul. Wileńska',
'ul. Wiśniowa',
'ul. Wincentego Witosa',
'ul. Wojska Polskiego',
'ul. Wrocławska',
'ul. Wschodnia',
'ul. Wspólna',
'ul. Józefa Wybickiego',
'ul. Zakładowa',
'ul. Zamkowa',
'ul. Zielona'
]

In [8]:
# zmieniam polskie znaki w nazwach ulic
for i in range(len(ulice)):
    ulice[i] = ulice[i].replace('ą','a')
    ulice[i] = ulice[i].replace('ć','c')
    ulice[i] = ulice[i].replace('ę','e')
    ulice[i] = ulice[i].replace('ł','l')
    ulice[i] = ulice[i].replace('ń','n')
    ulice[i] = ulice[i].replace('ó','o')
    ulice[i] = ulice[i].replace('ś','s')
    ulice[i] = ulice[i].replace('ż','z')
    ulice[i] = ulice[i].replace('ź','z')

## Dane

In [10]:
names_male = pd.read_csv('./dane/names_male.csv', names = ['names'], encoding='unicode_escape')
names_female = pd.read_csv('./dane/names_female.csv', names = ['names'], encoding='unicode_escape')
lastnames_male = pd.read_csv('./dane/lastnames_male.csv',names = ['names'], encoding='unicode_escape')
lastnames_female = pd.read_csv('./dane/lastnames_female.csv',names = ['names'], encoding='unicode_escape')
miasta = ["Wrocław", "Oleśnica", "Opole"]
names_male = (names_male.values).tolist() 
lastnames_male = (lastnames_male.values).tolist()
names_female = (names_female.values).tolist() 
lastnames_female = (lastnames_female.values).tolist()

## Schronisko

In [11]:
actual_time = time.time()
#tutaj zbieram dane o pracownikach
pr_id = []
o_years = 3.21 #lata działania schroniska

years2sec = 365.25*24*3600
#data otwarcia schroniska
open_time = actual_time - o_years*years2sec
zwolnienia  = []

## Pracownicy

### Byli opiekunowie

In [31]:
byli_opiekunowie = [] 
opiekunowie = []
for i in range(4):
    p_id = id_generator(i=i, first=2)
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wrocław")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wrocław")
    zatr = dataformat(open_time)
    zwol = opiekun["zwoln"]
    zwolnienia.append(zwol)
    zwo = dataformat(zwol)
    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': 'opiekun', 'pensja': 2500}
    byli_opiekunowie.append(opiekun_d)
    opiekunowie.append(opiekun_d)
byli_opiekunowie = pd.DataFrame(data=byli_opiekunowie)
byli_opiekunowie

,adres,id,imie,nazwisko,pensja,stanowisko,telefon,zatrudnienie,zwolenienie
0,ul. Osiedle 21 Wrocław,20000,ANNA,DAKHIL,2500,opiekun,175-619-619,2016-04-04,2018-11-10
1,ul. Przyjazni 76 Wrocław,20001,KAROLINA,DÅ»EDÅ»,2500,opiekun,167-276-302,2016-04-04,2018-01-13
2,ul. Boczna 32 Wrocław,20002,DANIEL,ZDENKOWSKI,2500,opiekun,990-871-456,2016-04-04,2018-04-01
3,ul. Fabryczna 90 Wrocław,20003,IZABELA,RAMOLA-PIÃRKOWSK,2500,opiekun,616-594-579,2016-04-04,2017-10-11


### Aktualni opiekunowie

In [32]:
#najpierw opiekunowie
#4 opiekunow pracuje aktualnie
for i in range(4):
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wrocław")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wrocław")

    zatr = dataformat(zwolnienia[i]+np.random.uniform(1,30)*24*3600)

    zwo = "9999-99-99"
    p_id = id_generator(i=i, first=2)
    pr_id.append(p_id)
    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': 'opiekun', 'pensja': 2500}
    opiekunowie.append(opiekun_d)


### Reszta pracowników

In [33]:
#kierowca, ksiegowy, kierownik, asystent kierownika -pracuja od poczatku
pensje = ['3000','3500','5000', '4000']
stanowiska = ['kierowca', 'księgowy', 'kierownik', 'asystent kierownika']
for i in range(4):
    p_id = id_generator(i=i, first=1)
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wrocław")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wrocław")
    zatr = dataformat(open_time)
    zwo = "9999-99-99"


    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': stanowiska[i], 'pensja': pensje[i]}
    opiekunowie.append(opiekun_d)


In [34]:
pd.DataFrame(data=opiekunowie)

,adres,id,imie,nazwisko,pensja,stanowisko,telefon,zatrudnienie,zwolenienie
0,ul. Osiedle 21 Wrocław,20000,ANNA,DAKHIL,2500,opiekun,175-619-619,2016-04-04,2018-11-10
1,ul. Przyjazni 76 Wrocław,20001,KAROLINA,DÅ»EDÅ»,2500,opiekun,167-276-302,2016-04-04,2018-01-13
2,ul. Boczna 32 Wrocław,20002,DANIEL,ZDENKOWSKI,2500,opiekun,990-871-456,2016-04-04,2018-04-01
3,ul. Fabryczna 90 Wrocław,20003,IZABELA,RAMOLA-PIÃRKOWSK,2500,opiekun,616-594-579,2016-04-04,2017-10-11
4,ul. Łuzycka 95 Wrocław,20000,JANUSZ,ZEQO,2500,opiekun,177-849-300,2019-02-25,9999-99-99
5,ul. Wikliniarska 14 Wrocław,20001,DANIEL,GISLER,2500,opiekun,927-654-235,2017-10-16,9999-99-99
6,ul. Poprzeczna 33 Wrocław,20002,ANTONI,BIN ZAINUDIN,2500,opiekun,578-724-773,2018-07-20,9999-99-99
7,ul. Zakladowa 31 Wrocław,20003,MIKOÅA,TAJER,2500,opiekun,483-970-525,2019-02-11,9999-99-99
8,ul. Brzozowa 86 Wrocław,10000,LENA,PACHWALD,3000,kierowca,479-872-227,2016-04-04,9999-99-99
9,ul. Stanislawy Walasiewicz 87 Wrocław,10001,JOANNA,BUKRAB,3500,księgowy,587-812-337,2016-04-04,9999-99-99


### Wolontariusze

In [35]:
#przez schronisko przewinelo sie 67 wolontariuszy
wolontariusze = []
for i in range(67):
    p_id = id_generator(i=i, first=4)
        
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, "Wrocław")
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, "Wrocław")
        
    zatr = dataformat(opiekun['zatr'])
    
    
    zwol = opiekun["zwoln"]
    zwo = dataformat(zwol)
    
    if np.random.random() > 0.9:
        zwo = '9999-99-99'
    
    wolontariusz =  {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, 
                     "rezygnacja": zwo}
    wolontariusze.append(wolontariusz)


In [36]:
pd.DataFrame(data=wolontariusze)

,adres,id,imie,nazwisko,rezygnacja,telefon,zatrudnienie
0,ul. Bociania 86 Wrocław,40000,STANISÅAW,KOBSA,2019-06-01,825-671-402,2018-09-20
1,ul. Targowa 79 Wrocław,40001,LENA,GRABIÅ,2017-10-08,604-245-806,2017-05-19
2,ul. Zielona 39 Wrocław,40002,TOMASZ,GATZKA,2017-12-01,610-188-629,2017-07-18
3,ul. Henryka Sienkiewicza 90 Wrocław,40003,ANTONI,GOLAÅSK,2019-02-19,117-472-136,2018-07-19
4,ul. Otwarta 50 Wrocław,40004,ANETA,PITRACHA,2017-01-11,981-668-720,2016-08-13
5,ul. Henryka Łasaka 20 Wrocław,40005,GRAÅ»YN,MOLÄ,2019-02-06,895-383-140,2018-12-18
6,ul. Tadeusza Rejtana 11 Wrocław,40006,SÅAWOMI,GUDYNG,2018-01-04,764-632-713,2016-12-15
7,ul. Feliksa Stamma 51 Wrocław,40007,JANINA,BERRY ORÅOWSK,2019-04-10,265-419-110,2019-01-11
8,ul. Blekitna 95 Wrocław,40008,MATEUSZ,ANGEÅO,2019-06-04,688-526-573,2018-06-21
9,ul. Bratnia 68 Wrocław,40009,BEATA,MÄDRAL,2019-05-27,664-921-924,2019-05-26


### Weterynarze

In [37]:
# 3 weterynarzy
#najpierw opiekunowie
#4 opiekunow pracuje aktualnie
weterynarze =[] 
w_id = []
miastaW = ["Wrocław", "Wrocław", "Oleśnica","Wrocław"]
for i in range(4):
    rand = np.random.random()
    if rand>0.5:
        opiekun = pracownik(names_male, lastnames_male, ulice, miastaW[i])
    else:
        opiekun = pracownik(names_female, lastnames_female, ulice, miastaW[i])

    zatr = dataformat(open_time)
    zwo = "9999-99-99"
    p_id = id_generator(i=i, first=3)
    w_id.append(p_id)
    opiekun_d = {'id':p_id, 'imie': opiekun['name'], 'nazwisko': opiekun['last_name'], 
               'adres': opiekun['adres'], 'telefon': opiekun['telefon'], 'zatrudnienie':zatr, "zwolenienie": zwo,
          'stanowisko': 'weterynarz'}
    weterynarze.append(opiekun_d)


In [38]:
pd.DataFrame(data=weterynarze)

,adres,id,imie,nazwisko,stanowisko,telefon,zatrudnienie,zwolenienie
0,ul. Boleslawa Prusa 78 Wrocław,30000,MIKOÅA,MARECHAL,weterynarz,909-614-879,2016-04-04,9999-99-99
1,ul. Kazimierza Deyny 41 Wrocław,30001,MARIAN,PAPST,weterynarz,635-597-457,2016-04-04,9999-99-99
2,ul. Pogodna 90 Oleśnica,30002,JAKUB,DUDERSTADT,weterynarz,529-192-253,2016-04-04,9999-99-99
3,ul. Lwowska 80 Wrocław,30003,ADAM,PAKÅO,weterynarz,767-210-531,2016-04-04,9999-99-99


# Zwierzęta

In [39]:
informacje = []
gatunki = []
zwierz_schr = []
zwierz_byle = []
adoptujacy = []
hist_zdr = []
transakcje = []

### gatunki

In [40]:
#gatunki
gatunki = {'nazwa' : ['pies', 'kot', 'szczur'], 'id_gatunku': [1,2,3]}
gat = pd.DataFrame(data=gatunki)
gat

,nazwa,id_gatunku
0,pies,1
1,kot,2
2,szczur,3


## W schronisku

### pieski

In [41]:
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 92
k = 1
l = 1
for i in range(len_dog):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'pies')
    birth = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 1
    dog_schr['id_gat'] = dog_info['id_gat']

    dog_info['id'] = id_generator(i=i, first=6, second =1)
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    if dog_info['masa']>=20:
        dog_info['ID opiekuna'] = [pr_id[0]]
        dog_info['blok'] = 'A_'+str(k)
        k += 1
    else:
        dog_info['ID opiekuna'] = [pr_id[1]]
        dog_info['blok'] = 'B_'+str(l)
        l += 1
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytuł']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    

### kotki

In [42]:
#92 psy- tyle liczy aktualnie nasze schronisko. 
len_cat = 41
k = 0
for i in range(len_cat):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'kot')
    birth = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 2
    dog_schr['id_gat'] = dog_info['id_gat']
    dog_info['id'] = id_generator(i=i, first=6, second =2)
        
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    dog_info['ID opiekuna'] = [pr_id[3]]
    if i%3==0:
        dog_info['blok'] = 'C_'+str(i//3)
        k = i
    else:
        dog_info['blok'] = 'C_'+str(k//3)
    
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytuł']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    

### szczurki

In [43]:
#92 psy- tyle liczy aktualnie nasze schronisko. 
len_rat = 17
k=0
for i in range(len_rat):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'szczur')
    birth = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 2
    dog_schr['id_gat'] = dog_info['id_gat']
    
    dog_info['id'] = id_generator(i=i, first=6, second =3)
        
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    dog_info['ID opiekuna'] = [pr_id[3]]
    dog_info['blok'] = 'C_14'
    
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytuł']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    

In [44]:
pd.DataFrame(transakcje)

,data,id_transakcji,imie,kwota,nazwisko,tytuł
0,2018-10-21,52000,ADAM,65,PAKÅO,"weterynarz, id_zwierzecia:61000"
1,2018-03-09,51001,ADAM,137,PAKÅO,"weterynarz, id_zwierzecia:61001"
2,2019-05-24,51002,MARIAN,51,PAPST,"weterynarz, id_zwierzecia:61002"
3,2019-04-04,51003,JAKUB,150,DUDERSTADT,"weterynarz, id_zwierzecia:61003"
4,2018-12-28,52004,MARIAN,127,PAPST,"weterynarz, id_zwierzecia:61004"
5,2019-03-25,52005,ADAM,136,PAKÅO,"weterynarz, id_zwierzecia:61005"
6,2019-03-11,51006,ADAM,80,PAKÅO,"weterynarz, id_zwierzecia:61006"
7,2018-12-17,51007,MIKOÅA,121,MARECHAL,"weterynarz, id_zwierzecia:61007"
8,2017-12-24,51008,ADAM,53,PAKÅO,"weterynarz, id_zwierzecia:61008"
9,2019-02-23,52009,MARIAN,88,PAPST,"weterynarz, id_zwierzecia:61009"


In [45]:
pd.DataFrame(hist_zdr)

,ID pracownika,ID_wizyty,data wizyty,diagnoza,id,stan zdrowia,sterylizacja,zalecenia
0,30003,42000,2018-10-21,inne,61000,zdrowy,tak,lek
1,30003,41001,2018-03-09,przeziębienie,61001,zdrowy,tak,lek
2,30001,41002,2019-05-24,przeziębienie,61002,zdrowy,nie,lek
3,30002,41003,2019-04-04,skaleczenie,61003,zdrowy,tak,lek
4,30001,42004,2018-12-28,zapalenie uszu,61004,zdrowy,tak,lek
5,30003,42005,2019-03-25,zapalenie spojówek,61005,zdrowy,tak,lek
6,30003,41006,2019-03-11,skaleczenie,61006,zdrowy,tak,lek
7,30000,41007,2018-12-17,inne,61007,zdrowy,tak,lek
8,30003,41008,2017-12-24,skaleczenie,61008,zdrowy,tak,lek
9,30001,42009,2019-02-23,skaleczenie,61009,zdrowy,tak,lek


In [46]:
pd.DataFrame(zwierz_schr)

,ID opiekuna,Stan,blok,id,id_gat
0,[20005],nadaje się do adopcji,B_1,61000,1
1,[20004],nadaje się do adopcji,A_1,61001,1
2,[20004],nadaje się do adopcji,A_2,61002,1
3,[20005],nadaje się do adopcji,B_2,61003,1
4,[20005],nadaje się do adopcji,B_3,61004,1
5,[20005],nadaje się do adopcji,B_4,61005,1
6,[20005],nadaje się do adopcji,B_5,61006,1
7,[20005],nadaje się do adopcji,B_6,61007,1
8,[20005],nadaje się do adopcji,B_7,61008,1
9,[20005],nadaje się do adopcji,B_8,61009,1


In [47]:
pd.DataFrame(informacje)

,ID opiekuna,blok,data_przyjecia,data_urodzenia,gatunek,id,id_gat,imie,inne_zwierzeta,masa,plec,umaszczenie,wcześniej adoptowany
0,[20005],B_1,2016-06-11,2007-08-18,pies,61000,1,Jarys,łagodny,17.065,samiec,szary,nie
1,[20004],A_1,2016-07-22,2010-10-04,pies,61001,1,Ringo,łagodny,28.187,samiec,biało-rudy,nie
2,[20004],A_2,2018-09-08,2007-12-08,pies,61002,1,Bond,agresywny,25.396,samiec,biszkoptowy,nie
3,[20005],B_2,2018-04-03,2010-01-20,pies,61003,1,Penelopa,łagodny,19.099,samica,szary,nie
4,[20005],B_3,2017-05-26,2016-04-07,pies,61004,1,Olaf,łagodny,12.098,samiec,biało-szary,nie
5,[20005],B_4,2017-11-13,2004-10-15,pies,61005,1,Lakina,łagodny,14.187,samica,czarny,nie
6,[20005],B_5,2018-09-20,2005-02-16,pies,61006,1,Eliot,łagodny,17.173,samiec,szary,nie
7,[20005],B_6,2018-01-10,2007-09-22,pies,61007,1,Bond,łagodny,13.482,samiec,biało-brązowy,nie
8,[20005],B_7,2017-08-27,2010-10-08,pies,61008,1,Werka,łagodny,16.459,samica,czarny,nie
9,[20005],B_8,2018-08-02,2006-05-22,pies,61009,1,Cyrus,łagodny,6.203,samiec,biało-rudy,nie


## Zaadoptowane

### pieski

In [ ]:
#92 psy- tyle liczy aktualnie nasze schronisko, 41 kotow, i 17 szczurow
len_dog = 92
len_ad = 139
k = 1
l = 1
for i in range(len_dog, len_dog+len_ad):
    dog_info = {}
    dog_schr = {}
    dog_wiz = {}
    dog_trans = {}
    dog = Animal(species = 'pies')
    birth = dog.birth()
    birth_d = dataformat(birth)
    acc = dog.acceptance()
    if np.random.random()>0.7:
        dog_info['wcześniej adoptowany'] = "tak"
        adop = dog.adoption()
        #ponowne przyjecie
        acc = dog.next_acceptance()
        if np.random.random()>0.9:
            adop = dog.adoption()
            #ponowne przyjecie
            acc = dog.next_acceptance()
    else:
        dog_info['wcześniej adoptowany'] = "nie"
        
    adoption = dog.adoption()

    dog_info['data_urodzenia'] = birth_d
    dog_info['data_przyjecia'] = dataformat(acc)
    
    dog_info['plec'] = dog.plec()
    dog_info['masa'] = dog.masa()
    dog_info['imie'] = dog.name('./dane/npieski.csv')
    kastracja = dog.kastracja()
    dog_info['id_gat'] = 1
    dog_schr['id_gat'] = dog_info['id_gat']
    if i<10:
        dog_info['id'] = '6100'+str(i)
    else:
        dog_info['id'] = '610'+str(i)
        
    dog_schr['id'] = dog_info['id']
        
    if np.random.random()>0.9:
        dog_info['inne_zwierzeta'] = "agresywny"
    else:
        dog_info['inne_zwierzeta'] = "łagodny"
    umaszczenie= dog.umaszczenie()
    
    dog_info['umaszczenie']=umaszczenie[0]
    dog_info['gatunek'] = umaszczenie[1]
    
    if dog_info['masa']>=20:
        dog_info['ID opiekuna'] = [pr_id[0]]
        dog_info['blok'] = 'A_'+str(k)
        k += 1
    else:
        dog_info['ID opiekuna'] = [pr_id[1]]
        dog_info['blok'] = 'B_'+str(l)
        l += 1
    dog_schr['ID opiekuna'] = dog_info['ID opiekuna']
    dog_schr['blok'] = dog_info['blok']
    ## czy nadaje sie do adopcji?
    if np.random.random()>0.1:
        data_wiz, chor, zalec, cena = dog.Wizyty()
        if np.random.random()>0.7:
            data_wiz1, chor1, zalec1, cena1 = dog.Wizyty()
    zdrowie = dog.stan_zdrowia()
    if zdrowie  == "zdrowy":
        dog_schr['Stan'] = "nadaje się do adopcji"
    else:
        dog_schr['Stan'] = "nie nadaje się do adopcji - "+str(zdrowie)
    wizyty, daty, ceny = dog.choroby()
    for j in range(len(wizyty)):
        dog_wiz['ID_wizyty'] =  id_generator(i = i, first = 4, second = j)
        dog_wiz['id'] = dog_info['id']
        index =int(np.round(np.random.uniform(0,3.49),0))
        id_pr =  w_id[index]
        dog_wiz['ID pracownika'] = id_pr
        dog_wiz['data wizyty'] = dataformat(daty[j])
        dog_wiz['diagnoza'] = wizyty[j]
        dog_wiz['zalecenia'] = 'lek'
        dog_wiz['stan zdrowia'] = zdrowie
        dog_wiz['sterylizacja'] = kastracja[0]
        dog_trans['id_transakcji'] = id_generator(i = i, first = 5, second = j)
        dog_trans['tytuł']='weterynarz, id_zwierzecia:'+ str(dog_info['id'])
        dog_trans['kwota'] = ceny[j]
        dog_trans['data'] = dataformat(daty[j])
        dog_trans['imie'] = weterynarze[index]['imie']
        dog_trans['nazwisko'] = weterynarze[index]['nazwisko']
    transakcje.append(dog_trans)
    hist_zdr.append(dog_wiz)
    zwierz_schr.append(dog_schr)
    informacje.append(dog_info)
    
    